# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver


Links para a documentação:
- https://pypi.org/project/selenium/
- https://www.selenium.dev/documentation/en/

In [4]:
# pip freeze verifica as bibliotecas no computador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
## utilizada para a interface com a internet. Ela funciona como um robô que clica, insere dados, etc

nav = webdriver.Chrome() # indica que ecolhemos o chrome pra usar
##permitirá armazenar e transitar nos diferentes campos que encontraremos nas páginas WEB


- para ser usado o webdriver deve ser baixado no link abaixo
- https://sites.google.com/a/chromium.org/chromedriver/downloads
- entao o arquivo chromedriver.exe deve ser colocado denro da mesma pasta que esta o python.exe


In [5]:
# pesquisar cotação dolar

nav.get("https://www.google.com/") # get entra no site indicado
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
## com o inspecionar copiar o xpath do elemento que queremos, que no caso é a barra de pesquisa
## xpath é o codigo/endereço html daquele elemento  
## com send.keys enviar o que deve ser pesquisado

nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
## comando para clicar no enter e fazer a pesquisa

cotacao_dolar = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
## com a pesquia concluida, agora temos que pegar o valor
## usamos o mesmo metodo de antes, mas em vez de send_keys pra envir algo se usa o get_attribute para coletar o valor que esta no codigo
print(cotacao_dolar)

# pesquisar cotação dolar
## mesmo procedimento de antes, mas é bom fazer manualmente e verificar se a estrutura e os codigos xpath sao iguais
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# pegar cotacao ouro
nav.get("https://www.melhorcambio.com/") #acessa a pagina indicada
aba_original = nav.window_handles[0] # metodo que permite identificar a aba do navegador
nav.find_element_by_xpath('//*[@id="commodity-hoje"]/tbody/tr[2]/td[2]/a/img').click() 
#encontra o elemento no html da pagina que queremos e clica nele com metodo click

aba_nova = nav.window_handles[1] #aba aberta ao clicar no icone da cotaçao do ouro
nav.switch_to.window(aba_nova) #metodo para trocar de aba
cotacao_ouro = nav.find_element_by_id('comercial').get_attribute('value') #ja na pagina correta, seleciona por id e pega o valor
cotacao_ouro = cotacao_ouro.replace(",", ".") #troca a , por . pois o python identifica . para numeros decimais
print(cotacao_ouro)

nav.quit()

5.3633999999999995
6.533318442000001
325.04


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [6]:
import pandas as pd

produtos_df = pd.read_excel("Produtos.xlsx")
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [7]:
#o processo abaixo sera usado para cada moeda com suas respctivas variaveis
#loc acha no df produtos_df na coluna cotaçao, as linhas que possuem o valor dolar nacoluna moeda
# entao em cotaçao armazena os valores de cotacao_dolar
produtos_df.loc[produtos_df['Moeda']=="Dólar", "Cotação"] = float(cotacao_dolar)
produtos_df.loc[produtos_df['Moeda']=="Euro", "Cotação"] = float(cotacao_euro)
produtos_df.loc[produtos_df['Moeda']=="Ouro", "Cotação"] = float(cotacao_ouro)

#as outras colunas nao se alteraram entao é necessario atualiza-las
produtos_df['Preço Base Reais'] = produtos_df['Cotação'] * produtos_df['Preço Base Original']
produtos_df['Preço Final'] = produtos_df['Ajuste'] * produtos_df['Preço Base Reais']
# produtos_df["Preço Final"] = produtos_df["Preço Final"].map("{:.2f}".format)
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.363400,5363.346366,1.40,7508.684912
1,Carro Renault,4500.00,Euro,6.533318,29399.932989,2.00,58799.865978
2,Notebook Dell,899.99,Dólar,5.363400,4827.006366,1.70,8205.910822
3,IPhone,799.00,Dólar,5.363400,4285.356600,1.70,7285.106220
4,Carro Fiat,3000.00,Euro,6.533318,19599.955326,1.90,37239.915119
5,Celular Xiaomi,480.48,Dólar,5.363400,2577.006432,2.00,5154.012864
6,Joia 20g,20.00,Ouro,325.040000,6500.800000,1.15,7475.920000


### Agora vamos exportar a nova base de preços atualizada

In [8]:
produtos_df.to_excel("Produtos Atualizado.xlsx", index=False)